# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Database Management

**Subtopic** - Basic CRUD Operations with a Python SQLite Database

**Use Case** - Design a rollback mechanism for batch database operations that automatically reverts all changes if any part of the transaction fails.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

**Problem Overview**
You are tasked with implementing a rollback-safe batch operation executor for a transactional database simulation. Each operation modifies a simple in-memory table of user records. The supported operations include `INSERT`, `UPDATE`, and `DELETE`. If any operation in the batch fails (e.g., due to a constraint violation), the entire batch must be rolled back and no changes should be applied.

The system must simulate the state of the table as a dictionary with user IDs as keys and user names as values. Your task is to return the final state of the table if all operations succeed, or return the initial state (unchanged) if any operation fails.


**Input Format**

* A dictionary `initial_state` representing the current table state, where the key is the integer user ID and the value is a string user name.
* A list of strings `operations` representing a batch of operations. Each operation is one of the following:

  * `"INSERT id name"` - Adds a new user. Fails if the ID already exists.
  * `"UPDATE id name"` - Changes the name of an existing user. Fails if the ID does not exist.
  * `"DELETE id"` - Removes a user. Fails if the ID does not exist.
  * `"FAIL"` - An explicit failure trigger.


**Output Format**

* A dictionary representing the final state of the table if all operations succeed.
* The original `initial_state` if any operation fails.


**Examples**

**Example 1**
Input:
initial_state = {1: "Alice"}
operations = ["INSERT 2 Bob", "UPDATE 1 Alicia", "DELETE 2"]
Output:
{1: "Alicia"}
**Explanation**:
User 2 is added, User 1 is updated, then User 2 is deleted. All succeed.

**Example 2**
Input:
initial_state = {1: "Alice"}
operations = ["INSERT 1 Bob", "UPDATE 1 Alicia"]
Output:
{1: "Alice"}
**Explanation**:
`INSERT 1 Bob` fails because ID 1 already exists. Entire batch is rolled back.

**Example 3**
Input:
initial_state = {}
operations = ["INSERT 1 Alice", "FAIL", "DELETE 1"]
Output:
{}
**Explanation**:
A `"FAIL"` operation forces rollback even though other operations are valid.


# Requirements


**Explicit Requirements**

* The input dictionary must not be modified if the batch fails.
* The output must reflect all valid changes if no failure occurs.
* `INSERT` fails if the user ID already exists.
* `UPDATE` and `DELETE` fail if the user ID does not exist.
* The string `"FAIL"` is always considered a failure.


**Implicit Requirements**

* The original dictionary must be deeply copied to preserve rollback behavior.
* Operations must be applied in the order given.
* The name in `INSERT` or `UPDATE` can contain only one word (no spaces).
* User IDs are always integers and names are non-empty strings.


**Edge Cases**

* Empty operations list: returns a copy of the original state.
* All operations succeed: return full updated state.
* First or last operation is `"FAIL"`: triggers full rollback.
* Repeated operations on the same ID (e.g., insert then delete) are allowed if all valid.
* Attempt to `UPDATE` or `DELETE` non-existent user: triggers rollback.


**Function Signatures**
**Python**:
`def execute_batch(initial_state: dict[int, str], operations: list[str]) -> dict[int, str]:`


**Constraints**

* 0 <= len(operations) <= 10^5
* 0 <= len(initial_state) <= 10^5
* 0 <= user ID <= 10^9
* User names are lowercase or capitalized alphabetic strings with length 1 to 100
* Each operation string is well-formed and follows the allowed format
* `"FAIL"` is case-sensitive and always fails when encountered

In [ ]:
# code

import copy


def execute_batch(initial_state: dict[int, str], operations: list[str]) -> dict[int, str]:
    """
    Executes a batch of database operations on a user table. The table is
    represented as a dictionary with user ID as key and user name as value.

    If any operation fails due to a constraint violation or the explicit
    "FAIL" command, all changes are rolled back and the original state
    is returned.

    Parameters:
        initial_state (dict[int, str]): The original state of the user table.
        operations (list[str]): A list of string operations to apply.

    Returns:
        dict[int, str]: The final state after all operations if successful,
                        or the original state if any operation fails.
    """
    current_state = copy.deepcopy(initial_state)

    for operation in operations:
        parts = operation.strip().split()
        if not parts:
            return initial_state  # Empty operation is invalid

        command = parts[0]

        if command == "INSERT":
            if len(parts) != 3:
                return initial_state
            try:
                user_id = int(parts[1])
                name = parts[2]
            except (ValueError, IndexError):
                return initial_state
            if user_id in current_state:
                return initial_state
            current_state[user_id] = name

        elif command == "UPDATE":
            if len(parts) != 3:
                return initial_state
            try:
                user_id = int(parts[1])
                name = parts[2]
            except (ValueError, IndexError):
                return initial_state
            if user_id not in current_state:
                return initial_state
            current_state[user_id] = name

        elif command == "DELETE":
            if len(parts) != 2:
                return initial_state
            try:
                user_id = int(parts[1])
            except ValueError:
                return initial_state
            if user_id not in current_state:
                return initial_state
            del current_state[user_id]

        elif command == "FAIL":
            return initial_state

        else:
            return initial_state  # Unknown command triggers rollback

    return current_state

In [ ]:
# tests

"""Unit tests for the `execute_batch` rollback-safe executor."""
import unittest
from copy import deepcopy
from main import execute_batch   # assumes function lives in main.py


class TestExecuteBatch(unittest.TestCase):
    # 1. all operations succeed
    def test_success_simple_flow(self):
        init = {1: "alice"}
        ops = ["INSERT 2 bob", "UPDATE 1 alicia", "DELETE 2"]
        expected = {1: "alicia"}
        self.assertEqual(execute_batch(init, ops), expected)

    # 2. duplicate insert triggers rollback
    def test_insert_duplicate_id_rollback(self):
        init = {1: "alice"}
        ops = ["INSERT 1 bob"]
        self.assertEqual(execute_batch(init, ops), init)

    # 3. explicit FAIL rolls back
    def test_explicit_fail(self):
        init = {1: "alice"}
        ops = ["INSERT 2 bob", "FAIL", "DELETE 2"]
        self.assertEqual(execute_batch(init, ops), init)

    # 4. update missing id rolls back
    def test_update_missing_id(self):
        init = {}
        ops = ["UPDATE 1 alice"]
        self.assertEqual(execute_batch(init, ops), init)

    # 5. delete missing id rolls back
    def test_delete_missing_id(self):
        init = {1: "alice"}
        ops = ["DELETE 2"]
        self.assertEqual(execute_batch(init, ops), init)

    # 6. empty operation list returns untouched copy
    def test_empty_operations_returns_copy(self):
        init = {1: "alice"}
        result = execute_batch(init, [])
        self.assertEqual(result, init)
        self.assertIsNot(result, init)  # ensure deep copy

    # 7. insert then delete same id succeeds
    def test_insert_then_delete_same_id(self):
        init = {}
        ops = ["INSERT 1 alice", "DELETE 1"]
        self.assertEqual(execute_batch(init, ops), {})

    # 8. first command FAIL
    def test_first_command_fail(self):
        init = {1: "alice"}
        ops = ["FAIL", "UPDATE 1 alicia"]
        self.assertEqual(execute_batch(init, ops), init)

    # 9. unknown command triggers rollback
    def test_unknown_command(self):
        init = {}
        ops = ["UPSERT 1 alice"]
        self.assertEqual(execute_batch(init, ops), init)

    # 10. malformed operation string triggers rollback
    def test_malformed_insert_lacks_name(self):
        init = {}
        ops = ["INSERT 1"]
        self.assertEqual(execute_batch(init, ops), init)

    # 11. deep copy on success (mutating result doesn't alter original)
    def test_deep_copy_on_success(self):
        init = {1: "alice"}
        ops = ["UPDATE 1 alicia"]
        result = execute_batch(init, ops)
        result[2] = "bob"
        self.assertNotIn(2, init)

    # 12. multiple sequential valid operations
    def test_many_valid_operations(self):
        init = {}
        ops = [f"INSERT {i} user{i}" for i in range(1, 6)]
        expected = {i: f"user{i}" for i in range(1, 6)}
        self.assertEqual(execute_batch(init, ops), expected)


if __name__ == "__main__":
    unittest.main()



# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/9cb5f824-1d69-4b83-83fa-544c288b3d8a?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67561%2Fview

#### Model code:

```python
def execute_batch(initial_state: dict[int, str], operations: list[str]) -> dict[int, str]:
    # Create a deep copy of the initial state for safe operations
    current_state = initial_state.copy() # <Issue>: The use of initial_state.copy() makes a shallow copy. For a dictionary with primitive types (int -> str), this is acceptable. BUT: The problem explicitly says "The input dictionary must not be modified if the batch fails" and "The original dictionary must be deeply copied to preserve rollback behavior."
    
    for operation in operations:
        parts = operation.split()
        command = parts[0]
        
        if command == "INSERT":
            user_id = int(parts[1])
            name = parts[2]
            if user_id in current_state:
                return initial_state  # Fail and rollback
            current_state[user_id] = name
        
        elif command == "UPDATE":
            user_id = int(parts[1])
            name = parts[2]
            if user_id not in current_state:
                return initial_state  # Fail and rollback
            current_state[user_id] = name
        
        elif command == "DELETE":
            user_id = int(parts[1])
            if user_id not in current_state:
                return initial_state  # Fail and rollback
            del current_state[user_id]
        
        elif command == "FAIL":
            return initial_state  # Explicit fail and rollback
    
    return current_state

```